# instalación de paquetes y verificación

In [1]:
# comprobamos que la versión de python sea inferior a 3.8, ya que para usar tensorflow 1.15 se recomienda usar python
# 3.7 o inferior
import sys
import string
python_version=sys.version.split("(")[0]
if int(sys.version.split(".")[1])>7:
    print("you are using a python version higer than 3.7.x, please install python 3.7.x")
else:
    print("congrats, you are using {} which is compatible with this notebook".format(python_version))


congrats, you are using 3.7.9  which is compatible with this notebook


In [2]:
# si todo sale bien,el test debería correr y los resultados aparecerán debajo. Es normal si se salta alguno de los tests 
# que hay
!python3 models/research/object_detection/builders/model_builder_tf1_test.py


Running tests under Python 3.7.9: /home/bigdata/anaconda3/envs/test/bin/python3
[ RUN      ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params0 (True)
/home/bigdata/anaconda3/envs/test/lib/python3.7/site-packages/tensorflow_core/python/training/tracking/data_structures.py:669: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  if not isinstance(wrapped_dict, collections.Mapping):
[       OK ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params0 (True)
[ RUN      ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params1 (False)
[       OK ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params1 (False)
[ RUN      ] ModelBuilderTF1Test.test_create_experimental_model
[       OK ] ModelBuilderTF1Test.test_create_experimental_model
[ RUN      ] ModelBuilderTF1Test.test_create_faster_rcnn_from_config_with_crop_feat

# creación del dataset

In [7]:
# vamos a dividir nuestras imágenes y anotaciones en splits de train, test y validación.
# los argumentos -vr y -tr indican respectivamente los porcentajes de datos reservados para validación y test.
# el argumento -f indica el formato de las imágenes
!python3 scripts/train_test_split.py -i images_masks -o images_masks -tr 0.1 -vr 0.15 -f jpg
!tree -d images_masks

succesfully created validation split using:  15.0 % of data
succesfully created test split using:  10.0 % of data
succesfully created train split using:  75.0 % of data
images
├── test
├── train
└── val

3 directories


In [8]:
# vamos a convertir estas fotos en y anotaciones en datasets de formato COCO. Tenemos que hacerlo para los sets de train
# test y validación. Tenemos que pasar como argumento archivo txt con los labels siguiendo el formato indiado en la
# documentación oficial de labelme https://github.com/wkentaro/labelme/tree/master/examples/instance_segmentation. Un ejemplo
# de cómo debe ser este archivo se muestra debajo
!python3 scripts/labelme2coco.py images_masks/train images_masks/train/coco --labels training_masks/labels.txt
!python3 scripts/labelme2coco.py images_masks/test images_masks/test/coco --labels training_masks/labels.txt
!python3 scripts/labelme2coco.py images_masks/val images_masks/val/coco --labels training_masks/labels.txt
!tree -d images_masks

Creating dataset: images/train/coco
Generating dataset from: images/train/images_frame8050.json
Generating dataset from: images/train/images_frame3100.json
Generating dataset from: images/train/images_frame7900.json
Generating dataset from: images/train/images_frame5800.json
Generating dataset from: images/train/images_frame10250.json
Generating dataset from: images/train/images_frame5300.json
Generating dataset from: images/train/images_frame3750.json
Generating dataset from: images/train/images_frame10150.json
Generating dataset from: images/train/images_frame7350.json
Generating dataset from: images/train/images_frame8700.json
Generating dataset from: images/train/images_frame1350.json
Generating dataset from: images/train/images_frame3350.json
Generating dataset from: images/train/images_frame8500.json
Generating dataset from: images/train/images_frame1750.json
Generating dataset from: images/train/images_frame1550.json
Generating dataset from: images/train/images_frame9400.json
Ge

Generating dataset from: images/val/images_frame2300.json
Generating dataset from: images/val/images_frame9650.json
Generating dataset from: images/val/images_frame2050.json
Generating dataset from: images/val/images_frame9750.json
Generating dataset from: images/val/images_frame9700.json
Generating dataset from: images/val/images_frame9250.json
Generating dataset from: images/val/images_frame7000.json
Generating dataset from: images/val/images_frame9350.json
Generating dataset from: images/val/images_frame1300.json
Generating dataset from: images/val/images_frame5350.json
Generating dataset from: images/val/images_frame5550.json
Generating dataset from: images/val/images_frame950.json
Generating dataset from: images/val/images_frame8550.json
Generating dataset from: images/val/images_frame4300.json
Generating dataset from: images/val/images_frame9800.json
Generating dataset from: images/val/images_frame7300.json
Generating dataset from: images/val/images_frame3000.json
Generating data

<img src="labels_txt.PNG">

In [5]:
# ahora borramos las imágenes y anotaciones de las carpetas train, test y val, ya que la carpeta coco dentro de cada 
# una de estas carpetas ya contiene las imágenes originales
!rm images_masks/test/*
!mv images_masks/test/coco/* images_masks/test
!rm -rf images_masks/test/coco 

!rm images_masks/train/*
!mv images_masks/train/coco/* images_masks/train
!rm -rf images_masks/train/coco 

!rm images_masks/val/*
!mv images_masks/val/coco/* images_masks/val
!rm -rf images_masks/val/coco 

!tree -d images_masks

mv: cannot stat 'images/test/coco/*': No such file or directory
mv: cannot stat 'images/train/coco/*': No such file or directory
mv: cannot stat 'images/val/coco/*': No such file or directory
images
├── test
├── train
└── val

3 directories


In [6]:
# ahora tenemos que generar los tfrecords usando estos dataset en formato coco
!python3 models/research/object_detection/dataset_tools/create_coco_tf_record.py --logtostderr \
--train_image_dir=images_masks/train \
--val_image_dir=images_masks/val \
--test_image_dir=images_masks/test \
--train_annotations_file=images_masks/train/annotations.json \
--val_annotations_file=images_masks/val/annotations.json \
--testdev_annotations_file=images_masks/test/annotations.json \
--output_dir=training_masks/tfrecord \
--include_masks=True

Traceback (most recent call last):
  File "models/research/object_detection/dataset_tools/create_coco_tf_record.py", line 518, in <module>
    tf.app.run()
  File "/home/bigdata/anaconda3/envs/test/lib/python3.7/site-packages/tensorflow_core/python/platform/app.py", line 40, in run
    _run(main=main, argv=argv, flags_parser=_parse_flags_tolerate_undef)
  File "/home/bigdata/anaconda3/envs/test/lib/python3.7/site-packages/absl/app.py", line 300, in run
    _run_main(main, args)
  File "/home/bigdata/anaconda3/envs/test/lib/python3.7/site-packages/absl/app.py", line 251, in _run_main
    sys.exit(main(argv))
  File "models/research/object_detection/dataset_tools/create_coco_tf_record.py", line 498, in main
    remove_non_person_images=FLAGS.remove_non_person_images)
  File "models/research/object_detection/dataset_tools/create_coco_tf_record.py", line 392, in _create_tf_record_from_coco_annotations
    groundtruth_data = json.load(fid)
  File "/home/bigdata/anaconda3/envs/test/lib/pytho

# Entrenamiento y exportación del modelo

In [ ]:
# entrenamos el modelo usando estos tfrecords. El modelo resultante se guardará el carpeta training_masks
!python3 models/research/object_detection/model_main.py --alsologtostderr \
--model_dir=training_masks/ \
--pipeline_config_path=training_masks/mask_rcnn_inception_v2_coco.config \
--num_train_steps=400000

In [ ]:
# aquí nos vamos a fijar en los archivos model.ckpt y vamos a buscar el que tenga el número más alto, ya que este número
# indica el step del proceso de entrenamiento en el que se realizó el checkpoint del modelo
!dir training_masks

In [ ]:
# por si no lo ves claro, corriendo esto te dirá qué debes poner en el script siguiente que generará el grafo de inferencia
import glob
import os
max_step=0
for file in glob.glob("training_masks/*"):
    if ("model.ckpt" in  file):
        step=int (file.split(os.sep)[1].split(".")[1].split("-")[1])
        max_step= step if step>max_step else max_step
print("En el campo trained_checkpoint debes poner lo siguiente: \ntraining_masks/model.ckpt-"+str(max_step))

In [ ]:
# ahora vamos a exportar este modelo como un grafo de inferencia. Este grafo se podrá usar para generar predicciones luego.
# Debemos sustituir los parámetros de entrada ["pipeline_config_path","trained_checkpoint_prefix","output_directory"]
# por los nuestros. Es recomendable elegir un nombre reconocible para la carpeta de salida, como por ejemplo
# "inference_graph" seguido del modelo entrenado y la fecha y la hora
!python3 models/research/object_detection/export_inference_graph.py --input_type image_tensor \
--pipeline_config_path training_masks/mask_rcnn_inception_v2_coco.config \
--trained_checkpoint_prefix training_masks/model.ckpt-2744 \
--output_directory inference_graph_mask_rcnn_02_09_10_14

In [ ]:
!dir inference_graph_mask_rcnn_02_09_10_14

# Generando predicciones en imágenes

In [ ]:
!python3 scripts/predict_on_images.py \
-i images_masks/test \
-o predicions_image_masks \
-m inference_graph_02_09_10_14 \
-l training_masks/label_map.pbtxt
-f jpg